# Pandas Functions and Commands - Part 93

This notebook documents pandas functions and commands with examples.

## Type Checking Functions (continued)

In [ ]:
import pandas as pd
import numpy as np

### is_datetime64_any_dtype

Checks whether the provided array or dtype is of any datetime64 dtype.

In [ ]:
# Examples
print(pd.api.types.is_datetime64_any_dtype(str))
print(pd.api.types.is_datetime64_any_dtype(int))
print(pd.api.types.is_datetime64_any_dtype(np.datetime64))
print(pd.api.types.is_datetime64_any_dtype(pd.DatetimeTZDtype("ns", "US/Eastern")))
print(pd.api.types.is_datetime64_any_dtype(np.array(['a', 'b'])))
print(pd.api.types.is_datetime64_any_dtype(np.array([], dtype=np.datetime64)))
print(pd.api.types.is_datetime64_any_dtype(pd.DatetimeIndex([1, 2, 3], dtype=np.datetime64)))

### is_datetime64_dtype

Checks whether an array-like or dtype is of the datetime64 dtype.

In [ ]:
# Examples
print(pd.api.types.is_datetime64_dtype(object))
print(pd.api.types.is_datetime64_dtype(np.datetime64))
print(pd.api.types.is_datetime64_dtype(np.array([], dtype=int)))
print(pd.api.types.is_datetime64_dtype(np.array([], dtype=np.datetime64)))
print(pd.api.types.is_datetime64_dtype([1, 2, 3]))

### is_datetime64_ns_dtype

Checks whether the provided array or dtype is of the datetime64[ns] dtype specifically.

In [ ]:
# Examples
print(pd.api.types.is_datetime64_ns_dtype(str))
print(pd.api.types.is_datetime64_ns_dtype(int))
print(pd.api.types.is_datetime64_ns_dtype(np.datetime64))  # no unit
print(pd.api.types.is_datetime64_ns_dtype(pd.DatetimeTZDtype("ns", "US/Eastern")))
print(pd.api.types.is_datetime64_ns_dtype(np.array([], dtype=np.datetime64)))  # no unit
print(pd.api.types.is_datetime64_ns_dtype(np.array([], dtype="datetime64[ps]")))
print(pd.api.types.is_datetime64_ns_dtype(pd.DatetimeIndex([1, 2, 3], dtype=np.datetime64)))

### is_datetime64tz_dtype

Checks whether an array-like or dtype is of a DatetimeTZDtype dtype (timezone-aware datetime).

In [ ]:
# Examples
print(pd.api.types.is_datetime64tz_dtype(object))
print(pd.api.types.is_datetime64tz_dtype([1, 2, 3]))
print(pd.api.types.is_datetime64tz_dtype(pd.DatetimeIndex([1, 2, 3])))  # tz-naive
print(pd.api.types.is_datetime64tz_dtype(pd.DatetimeIndex([1, 2, 3], tz="US/Eastern")))

dtype = pd.DatetimeTZDtype("ns", tz="US/Eastern")
s = pd.Series([], dtype=dtype)
print(pd.api.types.is_datetime64tz_dtype(dtype))
print(pd.api.types.is_datetime64tz_dtype(s))

### is_extension_type

Check whether an array-like is of a pandas extension class instance.

**Note:** Deprecated since version 1.0.0: Use is_extension_array_dtype instead.

In [ ]:
# This function is deprecated, use is_extension_array_dtype instead
# Example of the recommended function
cat_series = pd.Series(pd.Categorical([1, 2, 3]))
print(pd.api.types.is_extension_array_dtype(cat_series))

## Extension API

Pandas provides an extension API that allows you to create custom data types and accessors.

### register_dataframe_accessor

A decorator for registering custom accessors for pandas DataFrames.

In [ ]:
# Example of creating a custom DataFrame accessor

@pd.api.extensions.register_dataframe_accessor("geo")
class GeoAccessor:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
        
    @property
    def center(self):
        # Return the geographic center point of this DataFrame
        lat = self._obj.latitude
        lon = self._obj.longitude
        return (float(lon.mean()), float(lat.mean()))
    
    def plot(self):
        # This would plot the data on a map (implementation not shown)
        print("Plotting geographic data...")
        
# Create a sample DataFrame with geographic data
ds = pd.DataFrame({
    'longitude': np.linspace(0, 10, 5),
    'latitude': np.linspace(0, 20, 5)
})

# Use the custom accessor
print(ds.geo.center)
ds.geo.plot()

### ExtensionDtype

A base class for creating custom data types to be paired with an ExtensionArray.

In [ ]:
# Example of a simple custom ExtensionDtype
from pandas.api.extensions import ExtensionDtype, ExtensionArray
import numpy as np

class MyStringDtype(ExtensionDtype):
    name = 'my_string'
    type = str
    na_value = None
    
    def construct_from_string(self, string):
        if string == self.name:
            return self
        else:
            raise TypeError(f"Cannot construct {self.name} from {string}")
    
    @classmethod
    def construct_array_type(cls):
        return MyStringArray

# This is a simplified example and would need more implementation
# to be fully functional as an ExtensionArray
class MyStringArray(ExtensionArray):
    pass

# Print information about our custom dtype
my_dtype = MyStringDtype()
print(f"Name: {my_dtype.name}")
print(f"Type: {my_dtype.type}")
print(f"NA value: {my_dtype.na_value}")

## Practical Applications

Let's see how these type checking functions can be used in real-world scenarios:

In [ ]:
# Create a DataFrame with various column types
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': pd.date_range('20210101', periods=3),
    'C': pd.date_range('20210101', periods=3, tz='US/Eastern'),
    'D': pd.Series([1, 2, 3], dtype='category'),
    'E': [True, False, True]
})

# Function to describe column types
def describe_column_type(series):
    if pd.api.types.is_numeric_dtype(series):
        return "Numeric"
    elif pd.api.types.is_datetime64_any_dtype(series):
        if pd.api.types.is_datetime64tz_dtype(series):
            return f"Datetime with timezone: {series.dt.tz}"
        else:
            return "Datetime (timezone naive)"
    elif pd.api.types.is_categorical_dtype(series):
        return f"Categorical with {series.nunique()} unique values"
    elif pd.api.types.is_bool_dtype(series):
        return "Boolean"
    else:
        return "Other type"

# Apply the function to each column
for col in df.columns:
    print(f"Column {col}: {describe_column_type(df[col])}")